In [ ]:
import socket
import threading
import cv2
import pickle
import struct

In [ ]:
s = socket.socket()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.connect(('192.168.99.1',3333)) # Use the ip you want to connect

In [ ]:
cap=cv2.VideoCapture(0) # 0 based on your camera
def recv():
    size_of_msg=struct.calcsize("L")
    data=b''
    
    while True:      
        while len(data) < size_of_msg:
            data += s.recv(1024)
            
        true_msg_size = data[:size_of_msg]
        data = data[size_of_msg:]
        msg_size = struct.unpack("L", true_msg_size)[0]
        
        while len(data) < msg_size:
            data += s.recv(4096)
            
        image_data = data[:msg_size]
        data = data[msg_size:]
        image = pickle.loads(image_data)
        ret1 , live1 = cap.read()
        frame1 = cv2.resize(live1, (240,240))
        image[240:,400:] = frame1
        cv2.imshow('in sender file of 1 camera', image)
        if cv2.waitKey(1) == 13:
            break
            
    cv2.destroyAllWindows()
    cap.release()
    
t1=threading.Thread(target=recv)
t1.start()

while True:
    ret , photo = cap.read() 
    image = pickle.dumps(photo)
    img_size = struct.pack("L", len(image)) 
    s.sendall(img_size + image)

cv2.destroyAllWindows()
cap.release()